# 94-775/95-865: Topic Modeling with Latent Dirichlet Allocation

Author: George H. Chen (georgechen [at symbol] cmu.edu)

The beginning part of this demo is a shortened and modified version of sklearn's LDA & NMF demo (http://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html).

We'll use NumPy.

In [1]:
import numpy as np
np.set_printoptions(precision=5, suppress=True)

## Latent Dirichlet Allocation

We first load in 10,000 posts from the 20 Newsgroups dataset.

In [2]:
from sklearn.datasets import fetch_20newsgroups
num_articles = 10000
data = fetch_20newsgroups(shuffle=True, random_state=0,
                          remove=('headers', 'footers', 'quotes')).data[:num_articles]

We can verify that there are 10,000 posts, and we can look at an example post.

In [3]:
len(data)

10000

In [6]:
type(data)

list

In [7]:
# you can take a look at what individual documents look like by replacing what index we look at
print(data[2])



Experimental Lyme Disease in Dogs Produces Arthritis and Persistant Infection,
The Journal of Infectious Diseases, March 1993, 167:651-664



We now fit a `CountVectorizer` model that will compute, for each post, what its raw word count histograms are (the "term frequencies" we saw in week 1).

The output of the following cell is the term-frequencies matrix, where rows index different posts/text documents, and columns index 1000 different vocabulary words. A note about the arguments to `CountVectorizer`:

- `max_df`: we only keep words that appear in at most this fraction of the documents
- `min_df`: we only keep words that appear in at least this many documents
- `stop_words`: whether to remove stop words
- `max_features`: among words that don't get removed due to the above 3 arguments, we keep the top `max_features` number of most frequently occuring words

In [8]:
vocab_size = 1000
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer does tokenization and can remove terms that occur too frequently, not frequently enough, or that are stop words

# document frequency (df) means number of documents a word appears in
tf_vectorizer = CountVectorizer(max_df=0.95,
                                min_df=2,
                                stop_words='english',
                                max_features=vocab_size)
tf = tf_vectorizer.fit_transform(data)

We can verify that there are 10,000 rows (corresponding to posts), and 1000 columns (corresponding to words).

In [9]:
tf.shape

(10000, 1000)

A note about the `tf` matrix: this actually is stored as what's called a sparse matrix (rather than a 2D NumPy array that you're more familiar with). The reason is that often these matrices are really large and the vast majority of entries are 0, so it's possible to save space by not storing where the 0's are.

In [10]:
type(tf)# to save space, matrix only store none 0 values in it. most value in matrix is 0

scipy.sparse.csr.csr_matrix

 To convert `tf` to a 2D NumPy table, you can run `tf.toarray()` (this does not modify the original `tf` variable).

In [11]:
type(tf.toarray())

numpy.ndarray

In [12]:
tf.toarray().shape

(10000, 1000)

We can figure out what words the different columns correspond to by using the `get_feature_names()` function; the output is in the same order as the column indices. In particular, we can index into the following list (i.e., so given a column index, we can figure out which word it corresponds to).

In [13]:
print(tf_vectorizer.get_feature_names())

['00', '000', '02', '03', '04', '0d', '0t', '10', '100', '11', '12', '128', '13', '14', '145', '15', '16', '17', '18', '19', '1990', '1991', '1992', '1993', '1d9', '1st', '1t', '20', '200', '21', '22', '23', '24', '25', '250', '26', '27', '28', '29', '2di', '2tm', '30', '300', '31', '32', '33', '34', '34u', '35', '36', '37', '38', '39', '3d', '3t', '40', '41', '42', '43', '44', '45', '46', '48', '50', '500', '55', '60', '64', '6um', '70', '75', '75u', '7ey', '80', '800', '86', '90', '91', '92', '93', '9v', 'a86', 'ability', 'able', 'ac', 'accept', 'access', 'according', 'act', 'action', 'actually', 'add', 'addition', 'address', 'administration', 'advance', 'age', 'ago', 'agree', 'ah', 'air', 'al', 'algorithm', 'allow', 'allowed', 'alt', 'america', 'american', 'analysis', 'anonymous', 'answer', 'answers', 'anti', 'anybody', 'apparently', 'appears', 'apple', 'application', 'applications', 'appreciate', 'appreciated', 'approach', 'appropriate', 'apr', 'april', 'archive', 'area', 'areas', 

We can also go in reverse: given a word, we can figure out which column index it corresponds to. To do this, we use the `vocabulary_` attribute.

In [15]:
tf_vectorizer.vocabulary_['book']

176

We can figure out what the raw counts are for the 0-th post as follows.

In [16]:
tf[0].toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 2, 3, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

We now fit an LDA model to the data.

In [17]:
num_topics = 10

from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=num_topics, learning_method='online', random_state=0)
lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

The fitting procedure determines the every topic's distribution over words; this information is stored in the `components_` attribute. There's a catch: we actually have to normalize to get the probability distributions (without this normalization, instead what the model has are pseudocounts for how often different words appear per topic).

In [18]:
lda.components_.shape

(10, 1000)

In [19]:
lda.components_.sum(axis=1)

array([ 36546.2359 ,  42637.17064,  49198.91281, 114036.11143,
        25195.09295,  38394.6771 ,  28563.10527,  47579.92792,
        39951.26973,  63662.21236])

In [20]:
topic_word_distributions = np.array([row / row.sum() for row in lda.components_])

We can verify that each topic's word distribution sums to 1.

In [21]:
topic_word_distributions.sum(axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

We can also print out what the probabilities for the different words are for a specific topic. This isn't very easy to interpret.

In [22]:
print(topic_word_distributions[0])

[0.00349 0.      0.      0.      0.      0.      0.      0.00305 0.00631
 0.      0.00113 0.      0.00012 0.00001 0.      0.00176 0.      0.
 0.      0.      0.00197 0.00141 0.00318 0.00074 0.      0.00227 0.
 0.0031  0.      0.      0.00002 0.      0.      0.00021 0.001   0.
 0.      0.      0.      0.      0.      0.00285 0.00329 0.      0.
 0.      0.      0.      0.0004  0.      0.      0.      0.      0.
 0.      0.00166 0.      0.      0.      0.      0.00041 0.      0.
 0.00617 0.00381 0.      0.00245 0.      0.      0.00103 0.00112 0.
 0.      0.00041 0.00124 0.      0.00207 0.00131 0.00144 0.0005  0.
 0.      0.      0.00059 0.      0.0002  0.      0.      0.      0.00084
 0.0021  0.00107 0.      0.      0.00008 0.00024 0.00128 0.00211 0.
 0.      0.      0.      0.      0.      0.00116 0.      0.0036  0.00287
 0.      0.      0.      0.      0.      0.      0.00038 0.00243 0.
 0.      0.      0.      0.      0.      0.      0.      0.00093 0.
 0.00089 0.00019 0.0003  0.      

Instead, usually people do something like looking at the most probable words per topic, and try to use these words to interpret what the different topics correspond to.

In [23]:
num_top_words = 20

print('Displaying the top %d words per topic and their probabilities within the topic...' % num_top_words)
print()

for topic_idx in range(num_topics):
    print('[Topic ', topic_idx, ']', sep='')
    sort_indices = np.argsort(-topic_word_distributions[topic_idx])
    for rank in range(num_top_words):
        word_idx = sort_indices[rank]
        print(tf_vectorizer.get_feature_names()[word_idx], ':', topic_word_distributions[topic_idx, word_idx])
    print()

Displaying the top 20 words per topic and their probabilities within the topic...

[Topic 0]
year : 0.03297118094738992
team : 0.019204666343880843
good : 0.016830138855415535
gun : 0.016178567835481206
new : 0.015603672962549789
mr : 0.01505081138184353
president : 0.01431129643505745
games : 0.014180220098801988
season : 0.012931131385430669
league : 0.010491378550696718
players : 0.010349770248161931
play : 0.010274396314227859
hockey : 0.009974003061194872
time : 0.009694174123867904
best : 0.009409285751708664
price : 0.009381674293086522
years : 0.009373344778562082
win : 0.009083391252969808
stephanopoulos : 0.008137787607873974
got : 0.007609708926778402

[Topic 1]
edu : 0.04029110605206494
file : 0.03401605842663014
com : 0.0222023046148782
ftp : 0.014891870249451676
available : 0.014872344410171648
program : 0.013900099687007832
files : 0.01346941290100786
mail : 0.012731186252528216
list : 0.012302522449955312
server : 0.012191608776669492
pub : 0.011781900926012206
send : 0

We can use the `transform()` function to figure out for each document, what fraction of it is explained by each of the topics.

In [24]:
doc_topic_matrix = lda.transform(tf)

In [25]:
doc_topic_matrix.shape

(10000, 10)

In [26]:
doc_topic_matrix[0]

array([0.00182, 0.00182, 0.00182, 0.00182, 0.00182, 0.00182, 0.00182,
       0.82791, 0.15754, 0.00182])

## Computing co-occurrences of words

Here, we count the number of newsgroup posts in which two words both occur. This part of the demo should feel like a review of co-occurrence analysis from earlier in the course, except now we use scikit-learn's built-in CountVectorizer. Conceptually everything else in the same as before.

In [ ]:
word1 = 'year'
word2 = 'team'

word1_column_idx = tf_vectorizer.vocabulary_[word1]
word2_column_idx = tf_vectorizer.vocabulary_[word2]

In [ ]:
np.array(tf.todense())

In [ ]:
tf[:, word1_column_idx].toarray()

In [ ]:
documents_with_word1 = (tf[:, word1_column_idx].toarray().flatten() > 0)

In [ ]:
documents_with_word2 = (tf[:, word2_column_idx].toarray().flatten() > 0)

In [ ]:
documents_with_both_word1_and_word2 = documents_with_word1 * documents_with_word2

Next, we compute the log of the conditional probability of word 1 appearing given that word 2 appeared, where we add in a little bit of a fudge factor in the numerator (in this case, it's actually not needed but some times you do have two words that do not co-occur for which you run into a numerical issue due to taking the log of 0).

In [ ]:
eps = 0.1
np.log2((documents_with_both_word1_and_word2.sum() + eps) / documents_with_word2.sum())

In [ ]:
def prob_see_word1_given_see_word2(word1, word2, vectorizer, eps=0.1):
    word1_column_idx = vectorizer.vocabulary_[word1]
    word2_column_idx = vectorizer.vocabulary_[word2]
    documents_with_word1 = (tf[:, word1_column_idx].toarray().flatten() > 0)
    documents_with_word2 = (tf[:, word2_column_idx].toarray().flatten() > 0)
    documents_with_both_word1_and_word2 = documents_with_word1 * documents_with_word2
    return np.log2((documents_with_both_word1_and_word2.sum() + eps) / documents_with_word2.sum())

## Topic coherence

The below code shows how one implements the topic coherence calculation from lecture.

In [ ]:
average_coherence = 0
for topic_idx in range(num_topics):
    print('[Topic ', topic_idx, ']', sep='')
    sort_indices = np.argsort(topic_word_distributions[topic_idx])[::-1]
    coherence = 0.
    for top_word_idx1 in sort_indices[:num_top_words]:
        word1 = tf_vectorizer.get_feature_names()[top_word_idx1]
        for top_word_idx2 in sort_indices[:num_top_words]:
            word2 = tf_vectorizer.get_feature_names()[top_word_idx2]
            if top_word_idx1 != top_word_idx2:
                coherence += prob_see_word1_given_see_word2(word1, word2, tf_vectorizer, 0.1)
    print('Coherence:', coherence)
    print()
    average_coherence += coherence
average_coherence /= num_topics
print('Average coherence:', average_coherence)

## Number of unique words

The below code shows how one implements the number of unique words calculation from lecture.

In [ ]:
average_number_of_unique_top_words = 0
for topic_idx1 in range(num_topics):
    print('[Topic ', topic_idx1, ']', sep='')
    sort_indices1 = np.argsort(topic_word_distributions[topic_idx1])[::-1]
    num_unique_top_words = 0
    for top_word_idx1 in sort_indices1[:num_top_words]:
        word1 = tf_vectorizer.get_feature_names()[top_word_idx1]
        break_ = False
        for topic_idx2 in range(num_topics):
            if topic_idx1 != topic_idx2:
                sort_indices2 = np.argsort(topic_word_distributions[topic_idx2])[::-1]
                for top_word_idx2 in sort_indices2[:num_top_words]:
                    word2 = tf_vectorizer.get_feature_names()[top_word_idx2]
                    if word1 == word2:
                        break_ = True
                        break
                if break_:
                    break
        else:
            num_unique_top_words += 1
    print('Number of unique top words:', num_unique_top_words)
    print()
    
    average_number_of_unique_top_words += num_unique_top_words
average_number_of_unique_top_words /= num_topics
print('Average number of unique top words:', average_number_of_unique_top_words)